# Quick Start

If you are familiar with using a REST based API already, you might find...

If you are not, we can recommend reading (prog hist) if you need to . Otherwise, you might want to take a look at our 
datasets which might cover the area of interest without needing to programme. 

## Most common operation 

  * Retrieve a known object
  * Search for one or more objects
  * Browse/Filter for one or more objects
  * Cluster fields from more than one objects



## Retrieve an object

The most simple of operations, if you already know the object you want to retrieve, retrieve it.

## Searching for object(s)

In [3]:
import requests
import pandas as pd
from ipyaggrid import Grid

df = pd.read_csv("http://vam-etc-test-api.azureedge.net/api/v2/objects/search?id_material=AAT45514&page_size=50&format=csv")

column_defs = [{'headerName':'System Number','field':'systemNumber','rowGroup':True},
               {'headerName':'Object Type','field':'objectType','rowGroup':True},
               {'headerName':'Place','field':'_primaryPlace'}
              ]

grid_options = {
    'columnDefs' : column_defs,
    'enableSorting': True,
    'enableFilter': True,
    'enableColResize': True,
    'enableRangeSelection': True,
}

g = Grid(grid_data=df, grid_options=grid_options)
display(g)

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': […

## Browse/Filter for one or more objects

## Cluster 

### Clustering objects

In [3]:
import pandas
from ipyaggrid import Grid
dfm = pandas.DataFrame( [[1,2,3],[4,5,6]],columns=['A', 'B', 'C'],)
column_defs = [{'field': dfm.index.name}] + [{'field': c} for c in dfm.columns[:3]]
grid_options = {
    'columnDefs' : column_defs,
}

g = Grid(grid_data=dfm,
         grid_options=grid_options,
         index=True)
display(g)

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='350px', menu={'buttons': […

In [4]:
import qgrid
import pandas
dfm = pandas.DataFrame( [[1,2,3],[4,5,6]],columns=['A', 'B', 'C'],)
qgrid_widget = qgrid.show_grid(dfm, show_toolbar=True)
display(qgrid_widget)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [5]:
pip install qgrid

     |████████████████████████████████| 889 kB 4.3 MB/s eta 0:00:01
  Created wheel for qgrid: filename=qgrid-1.3.1-py2.py3-none-any.whl size=1761254 sha256=a510cb8d19e86ec49f80b745c507dd855d19dd1f70662601639592618b80e198
  Stored in directory: /home/richard/.cache/pip/wheels/4a/65/d3/ec3d01337551a41a66aa7fe06c4d775b4a6a8ca798244a7e84
Successfully built qgrid
Note: you may need to restart the kernel to use updated packages.
